In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.utilities.cli import LightningCLI

from models import LSTMPredictor, PredictNextTimestep
from torchinfo import summary

In [2]:
model_simple = PredictNextTimestep(input_size = 5,
                    hidden_size = 128,
                    output_size = 5,
                    num_layers = 2,)

summary(model_simple, input_size = (32, 10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
PredictNextTimestep                      --                        --
├─LSTM: 1-1                              [32, 10, 128]             201,216
├─Linear: 1-2                            [32, 5]                   645
Total params: 201,861
Trainable params: 201,861
Non-trainable params: 0
Total mult-adds (M): 64.41
Input size (MB): 0.01
Forward/backward pass size (MB): 0.33
Params size (MB): 0.81
Estimated Total Size (MB): 1.14

In [3]:
model = LSTMPredictor("Dataset\IKCO1.csv",
                        learning_rate = 3e-4,
                        batch_size = 32,
                        split = [0.75, 0.25],
                        input_size = 5,
                        hidden_size = 128,
                        output_size = 5,
                        num_layers = 2,
                        timestep = 10,)


summary(model, input_size = (32, 10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
LSTMPredictor                            --                        --
├─PredictNextTimestep: 1-1               [32, 5]                   --
│    └─LSTM: 2-1                         [32, 10, 128]             201,216
│    └─Linear: 2-2                       [32, 5]                   645
├─MSELoss: 1-2                           --                        --
Total params: 201,861
Trainable params: 201,861
Non-trainable params: 0
Total mult-adds (M): 64.41
Input size (MB): 0.01
Forward/backward pass size (MB): 0.33
Params size (MB): 0.81
Estimated Total Size (MB): 1.14

In [4]:
checkpoint_callback = ModelCheckpoint(dirpath = 'my_model',
                                    every_n_epochs = 1,
                                    monitor = 'val_loss',
                                    mode = 'min')

early_stop_callback = EarlyStopping(monitor = "val_loss",
                                    min_delta = 0.00,
                                    patience = 3,
                                    verbose = False,
                                    mode = "min")

logger = TensorBoardLogger('pure-lstm-logs/', name = 'pure-lstm', version = 0)

In [5]:
trainer = Trainer(
    default_root_dir = 'pure-lstm-logs/',
    gpus = (1 if torch.cuda.is_available() else 0),
    callbacks = [checkpoint_callback],
    max_epochs = 10,
    logger = logger)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                | Params
----------------------------------------------
0 | model | PredictNextTimestep | 201 K 
1 | loss  | MSELoss             | 0     
----------------------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.807     Total estimated model params size (MB)


c:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:487: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
c:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Anaconda\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([1, 1, 5])) that is different to the input size (torch.Size([1, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Anacon

Epoch 9: 100%|██████████| 4077/4077 [01:07<00:00, 60.59it/s, loss=0.000877, v_num=0, train_loss_step=0.011, train_loss_epoch=0.000918]   


In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath = 'my_model1',
                                    every_n_epochs = 1,
                                    monitor = 'val_loss',
                                    mode = 'min')

early_stop_callback = EarlyStopping(monitor = "val_loss",
                                    min_delta = 0.00,
                                    patience = 3,
                                    verbose = False,
                                    mode = "min")

logger = TensorBoardLogger('pure-lstm-logs/', name = 'pure-lstm', version = 1)

In [ ]:
trainer = Trainer(
    default_root_dir = 'pure-lstm-logs/',
    gpus = (1 if torch.cuda.is_available() else 0),
    callbacks = [checkpoint_callback, early_stop_callback],
    max_epochs = 10,
    logger = logger)

trainer.fit(model)